In [13]:
import pandas as pd
from pyproj import CRS, Transformer
import os

In [ ]:
os.getcwd()

In [26]:
# Load the input CSV file
input_file = "ohio1.csv"  # Change this to your CSV file path
output_file = "ohio_out.csv"

# Read the CSV file
df = pd.read_csv(input_file, names=["latitude", "longitude", "height"])

# Define the CRS for NAD27 and NAVD88 (horizontal and vertical datums)
crs_nad27 = CRS.from_epsg(4267)  # EPSG:4267 = NAD27 (horizontal)
crs_navd88 = CRS.from_epsg(6319)  # EPSG:6319 = NAVD88 (vertical)

# Define the target CRS for EGM2008 geoid model (vertical)
crs_egm2008 = CRS.from_epsg(3855)  # EPSG:3855 = EGM2008 (vertical)

# Initialize the transformer for converting NAD27 (horizontal) to WGS84 (for modern transformations)
transformer_horiz = Transformer.from_crs(crs_nad27, CRS.from_epsg(4326), always_xy=True)  # NAD27 to WGS84

# Initialize the vertical transformation for NAVD88 to EGM2008
transformer_vert = Transformer.from_crs(crs_navd88, crs_egm2008, always_xy=True)  # NAVD88 to EGM2008

# Function to transform height with horizontal and vertical adjustments
def transform_coordinates(lat, lon, height):
    if pd.isna(lat) or pd.isna(lon) or pd.isna(height):
        return None  # Skip if any value is NaN
    try:
        # First, transform the horizontal coordinates from NAD27 to WGS84 (EPSG:4326)
        lon_wgs84, lat_wgs84 = transformer_horiz.transform(lon, lat)
        
        # Then, transform the height from NAVD88 to EGM2008 based on the corrected latitude and longitude
        height_egm2008 = transformer_vert.transform(lon_wgs84, lat_wgs84, height)
        return height_egm2008
    except Exception as e:
        return None  # Return None if the transformation fails

# Apply the transformation row by row
height_egm2008 = []
for _, row in df.iterrows():
    transformed_height = transform_coordinates(row["latitude"], row["longitude"], row["height"])
    height_egm2008.append(transformed_height)

# Add the new column to the DataFrame
df["height_egm2008"] = height_egm2008



In [27]:
# Save the result to a new CSV file
df.to_csv(output_file, index=False)
print(f"Transformation complete. Output saved to {output_file}")

Transformation complete. Output saved to ohio_out.csv
